### 下載 Nextstrain Zika 教程存儲庫
我們將病原體分析存儲在版本控制存儲庫中，因此我們可以輕鬆跟踪隨時間的變化。下載您要構建的示例 Zika 病原體存儲庫。
完成後，您將擁有一個名為zika-tutorial/.

In [1]:
## 導入環境變數 export PATH=...
import os
myENV='nextstrain'
myPackageHome='/home/ubuntu/miniconda3'
os.environ['PATH']=myPackageHome+"/envs/"+myENV+"/bin:"+os.environ['PATH'] 

In [2]:
#1
## 下載數據
!mkdir -p ~/nextstrain/
%cd ~/nextstrain/
!git clone https://github.com/nextstrain/avian-flu.git

/home/ubuntu/nextstrain
Cloning into 'avian-flu'...
remote: Enumerating objects: 288, done.
remote: Counting objects: 100% (102/102), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 288 (delta 61), reused 68 (delta 31), pack-reused 186
Receiving objects: 100% (288/288), 688.15 KiB | 1.46 MiB/s, done.
Resolving deltas: 100% (160/160), done.


## 準備序列, 請在左側檔案總管, 打開以下連結檔案
1. nextstrain/avian-flu/example_data/h5n1_ha.fasta

In [3]:
#2
## 建置metadata
%cd ~/nextstrain/avian-flu
!mkdir -p results/
!augur parse \
    --sequences example_data/h5n1_ha.fasta \
    --fields strain virus accession date region country city01 city02 other01 other02 other03 \
    --output-sequences results/sequences.fasta \
    --output-metadata results/metadata.tsv

/home/ubuntu/nextstrain/avian-flu


In [4]:
#3
## Index the Sequences
%cd ~/nextstrain/avian-flu
!augur index \
  --sequences results/sequences.fasta \
  --output results/sequence_index.tsv

/home/ubuntu/nextstrain/avian-flu


In [6]:
# 4
## Filter the Sequences
%cd ~/nextstrain/avian-flu
!augur filter \
  --sequences results/sequences.fasta \
  --sequence-index results/sequence_index.tsv \
  --metadata results/metadata.tsv \
  --output results/filtered.fasta \
  --min-date 2000 \
  --min-length 1700

/home/ubuntu/nextstrain/avian-flu
7 strains were dropped during filtering
	6 of these were dropped because of their date (or lack of date)
	1 of these were dropped because they were shorter than minimum length of 1700bp
26 strains passed all filters


In [7]:
#5 
## Align the Sequences
%cd ~/nextstrain/avian-flu
!augur align \
  --sequences results/filtered.fasta \
  --reference-sequence config/reference_h5n1_ha.gb \
  --output results/aligned.fasta \
  --fill-gaps

/home/ubuntu/nextstrain/avian-flu

using mafft to align via:
	mafft --reorder --anysymbol --nomemsave --adjustdirection --thread 1 results/aligned.fasta.to_align.fasta 1> results/aligned.fasta 2> results/aligned.fasta.log 

	Katoh et al, Nucleic Acid Research, vol 30, issue 14
	https://doi.org/10.1093%2Fnar%2Fgkf436

7bp insertion at ref position 0
	AGCAAAA: A/goose/Guangdong/xb/2001, A/Swine/Fujian/F1/2001, A/quail/Vietnam/CVVI_49/2010, A/Vietnam/UT36250_1/2010, A/wild_duck/Fujian/2/2011
	AGCGAAA: A/environment/Hunan/1_8/2007, A/chicken/Egypt/0813_NLQP/2008, A/environment/Hunan/1_35/2007
	TCAAAA: A/environment/Guizhou/4/2009
	AA: A/environment/Guizhou/9/2009
	AAA: A/water/Xinjiang/3/2009
1bp insertion at ref position 3
	G: A/environment/Guizhou/4/2009
12bp insertion at ref position 1760
	CTTGTTTCTACT: A/goose/Guangdong/xb/2001, A/Swine/Fujian/F1/2001, A/quail/Vietnam/CVVI_49/2010, A/environment/Hunan/1_8/2007, A/Vietnam/UT36250_1/2010, A/chicken/Egypt/0813_NLQP/2008, A/environment/Hun

In [8]:
#6
## Construct the Phylogeny
%cd ~/nextstrain/avian-flu
!augur tree \
    --alignment results/aligned.fasta \
    --output results/tree_raw.nwk \
    --method iqtree \
    --nthreads 2

/home/ubuntu/nextstrain/avian-flu
Building a tree via:
	iqtree2 -ninit 2 -n 2 -me 0.05 -nt 2 -s results/aligned-delim.fasta -m GTR  > results/aligned-delim.iqtree.log
	Nguyen et al: IQ-TREE: A fast and effective stochastic algorithm for estimating maximum likelihood phylogenies.
	Mol. Biol. Evol., 32:268-274. https://doi.org/10.1093/molbev/msu300


Building original tree took 0.1230630874633789 seconds


In [9]:
#7
## tree refine
%cd ~/nextstrain/avian-flu
!augur refine \
  --tree results/tree_raw.nwk \
  --alignment results/aligned.fasta \
  --metadata results/metadata.tsv \
  --output-tree results/tree.nwk \
  --output-node-data results/branch_lengths.json \
  --timetree \
  --coalescent const \
  --date-confidence \
  --date-inference marginal \
  --clock-filter-iqd 4

/home/ubuntu/nextstrain/avian-flu
augur refine is using TreeTime version 0.8.5

0.05	TreeTime.reroot: with method or node: least-squares

0.05	TreeTime.reroot: rerooting will ignore covariance and shared ancestry.

0.09	TreeTime.reroot: with method or node: least-squares

0.09	TreeTime.reroot: rerooting will ignore covariance and shared ancestry.
pruning leaf  AF144305.1

0.12	WARNING: Previous versions of TreeTime (<0.7.0) RECONSTRUCTED sequences of
    	tips at positions with AMBIGUOUS bases. This resulted in unexpected
    	behavior is some cases and is no longer done by default. If you want to
    	replace those ambiguous sites with their most likely state, rerun with
    	`reconstruct_tip_states=True` or `--reconstruct-tip-states`.

0.34	TreeTime.reroot: with method or node: least-squares

0.34	TreeTime.reroot: rerooting will account for covariance and shared ancestry.

0.54	###TreeTime.run: INITIAL ROUND

1.88	TreeTime.reroot: with method or node: least-squares

1.88	TreeTime.rer

In [ ]:
#8
## traits
%cd ~/nextstrain/avian-flu
!augur traits \
  --tree results/tree.nwk \
  --metadata results/metadata.tsv \
  --output-node-data results/traits.json \
  --columns region country \
  --confidence

In [11]:
#9
## ancestral
%cd ~/nextstrain/avian-flu
!augur ancestral \
  --tree results/tree.nwk \
  --alignment results/aligned.fasta \
  --output-node-data results/nt_muts.json \
  --inference joint

/home/ubuntu/nextstrain/avian-flu
augur ancestral is using TreeTime version 0.8.5

Inferred ancestral sequence states using TreeTime:
	Sagulenko et al. TreeTime: Maximum-likelihood phylodynamic analysis
	Virus Evolution, vol 4, https://academic.oup.com/ve/article/4/1/vex042/4794731

ancestral mutations written to results/nt_muts.json


In [12]:
#10
## translate
%cd ~/nextstrain/avian-flu
!augur translate \
  --tree results/tree.nwk \
  --ancestral-sequences results/nt_muts.json \
  --reference-sequence config/reference_h5n1_ha.gb \
  --output-node-data results/aa_muts.json

/home/ubuntu/nextstrain/avian-flu
Read in 2 features from reference sequence file
amino acid mutations written to results/aa_muts.json


In [14]:
#11
## export json
%cd ~/nextstrain/avian-flu
!augur export v2 \
  --tree results/tree.nwk \
  --metadata results/metadata.tsv \
  --node-data results/branch_lengths.json \
              results/traits.json \
              results/nt_muts.json \
              results/aa_muts.json \
  --colors config/colors_h5n1.tsv \
  --lat-longs config/lat_longs_h5n1.tsv \
  --auspice-config config/auspice_config_h5n1.json \
  --output auspice/avian-flu.json

/home/ubuntu/nextstrain/avian-flu
Validating schema of 'results/aa_muts.json'...
Validating config file config/auspice_config_h5n1.json against the JSON schema
Validating schema of 'config/auspice_config_h5n1.json'...





Validating produced JSON
Validating schema of 'auspice/avian-flu.json'...
Validating that the JSON is internally consistent...
Validation of 'auspice/avian-flu.json' succeeded, but there were warnings you may want to resolve.



In [15]:
#12
%cd ~/nextstrain/avian-flu
!wget https://covid-19.nchc.org.tw/nextstrain/config.json -O config.json
!auspice build --extend config.json

/home/ubuntu/nextstrain/avian-flu
--2021-12-14 21:47:25--  https://covid-19.nchc.org.tw/nextstrain/config.json
Resolving covid-19.nchc.org.tw (covid-19.nchc.org.tw)... 203.145.222.54
Connecting to covid-19.nchc.org.tw (covid-19.nchc.org.tw)|203.145.222.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 226 [application/json]
Saving to: ‘config.json’

config.json         100%[===================>]     226  --.-KB/s    in 0s      

2021-12-14 21:47:25 (20.7 MB/s) - ‘config.json’ saved [226/226]

Running webpack compiler
[BABEL] Note: The code generator has deoptimised the styling of /home/ubuntu/miniconda3/envs/nextstrain/lib/auspice/node_modules/lodash/lodash.js as it exceeds the max of 500KB.
[BABEL] Note: The code generator has deoptimised the styling of /home/ubuntu/miniconda3/envs/nextstrain/lib/auspice/node_modules/react-icons/fa/index.esm.js as it exceeds the max of 500KB.
[warning]	Webpack has warnings (run with '--verbose' to see them)


In [ ]:
#13
%cd ~/nextstrain/avian-flu
!export PORT=9999; export HOST=0.0.0.0; auspice view --datasetDir auspice

/home/ubuntu/nextstrain/avian-flu


---------------------------------------------------
Auspice server now running at http://0.0.0.0:9999
Serving the auspice build which exists in this directory.
Looking for datasets in /home/ubuntu/nextstrain/avian-flu/auspice
Looking for narratives in /home/ubuntu/miniconda3/envs/nextstrain/lib/auspice/node_modules/auspice/narratives
---------------------------------------------------


GET AVAILABLE returning locally available datasets & narratives
[warning]	Couldn't collect available narratives (path searched: /home/ubuntu/miniconda3/envs/nextstrain/lib/auspice/node_modules/auspice/narratives)
GET DATASET query received: prefix=/avian-flu
GET AVAILABLE returning locally available datasets & narratives
[warning]	Couldn't collect available narratives (path searched: /home/ubuntu/miniconda3/envs/nextstrain/lib/auspice/node_modules/auspice/narratives)
GET DATASET query received: prefix=/avian-flu&type=root-sequence
[warning]	Failed to read /home/ubuntu

In [20]:
#14
!lsof -i -P -n | grep `whoami` | grep LISTEN | grep 9999 | awk '{print $2}' | xargs kill